In [0]:
!pip install kaggle

In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


In [0]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
!kaggle competitions download -c histopathologic-cancer-detection -p /content/

  0% 0.00/1.33M [00:00<?, ?B/s]
100% 1.33M/1.33M [00:00<00:00, 42.9MB/s]
  0% 0.00/5.10M [00:00<?, ?B/s]
100% 5.10M/5.10M [00:00<00:00, 45.9MB/s]
 99% 1.29G/1.30G [00:11<00:00, 143MB/s]
100% 1.30G/1.30G [00:11<00:00, 122MB/s]
100% 4.98G/4.98G [02:49<00:00, 18.6MB/s]
100% 4.98G/4.98G [02:49<00:00, 31.5MB/s]


In [0]:
%cd /content
!ls

/content
adc.json     sample_submission.csv.zip	test.zip  train_labels.csv.zip
sample_data  test			train	  train.zip


In [0]:
!mkdir /content/test
!unzip -qq /content/test.zip -d /content/test
!rm /content/test.zip
!mkdir /content/train
!unzip -qq /content/train.zip -d /content/train
!rm /content/train.zip
!unzip -qq /content/train_labels.csv.zip
!rm /content/train_labels.csv.zip
!ls

mkdir: cannot create directory ‘/content/test’: File exists
mkdir: cannot create directory ‘/content/train’: File exists
adc.json  sample_data  sample_submission.csv.zip  test	train  train_labels.csv


In [0]:
import numpy as np
import pandas as pd

import gc
import os
from glob import glob
from random import shuffle
import cv2
import datetime
import matplotlib.pyplot as plt

from albumentations import Compose, RandomRotate90, Transpose, Flip, OneOf, CLAHE, IAASharpen, IAAEmboss, RandomBrightnessContrast, JpegCompression, Blur, GaussNoise, HueSaturationValue, ShiftScaleRotate, Normalize

from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.python.keras.layers import Lambda, Reshape, DepthwiseConv2D, ZeroPadding2D, Add, MaxPooling2D,Activation, Flatten, Conv2D, Dense, Input, Dropout, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import backend as K

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [0]:
from keras_applications import get_submodules_from_kwargs
from keras.applications import keras_modules_injection

backend = None
keras_utils = None


BASE_WEIGHTS_PATH = (
    'https://github.com/keras-team/keras-applications/'
    'releases/download/resnet/')
WEIGHTS_HASHES = {
    'resnet50': ('2cb95161c43110f7111970584f804107',
                 '4d473c1dd8becc155b73f8504c6f6626'),
    'resnext50': ('67a5b30d522ed92f75a1f16eef299d1a',
                  '62527c363bdd9ec598bed41947b379fc')
}

def block1(x, filters, kernel_size=3, stride=1,
           conv_shortcut=True, name=None):
    """A residual block.

    # Arguments
        x: input tensor.
        filters: integer, filters of the bottleneck layer.
        kernel_size: default 3, kernel size of the bottleneck layer.
        stride: default 1, stride of the first layer.
        conv_shortcut: default True, use convolution shortcut if True,
            otherwise identity shortcut.
        name: string, block label.

    # Returns
        Output tensor for the residual block.
    """
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    if conv_shortcut is True:
        shortcut = Conv2D(4 * filters, 1, strides=stride,
                                 name=name + '_0_conv')(x)
        shortcut = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                             name=name + '_0_bn')(shortcut)
    else:
        shortcut = x

    x = Conv2D(filters, 1, strides=stride, name=name + '_1_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                 name=name + '_1_bn')(x)
    x = Activation('relu', name=name + '_1_relu')(x)
    x = Conv2D(filters, kernel_size, padding='SAME',
                    name=name + '_2_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                 name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_relu')(x)

    x = Conv2D(4 * filters, 1, name=name + '_3_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_3_bn')(x)

    x = Add(name=name + '_add')([shortcut, x])
    x = Activation('relu', name=name + '_out')(x)
    return x


def stack1(x, filters, blocks, stride1=2, name=None):
    """A set of stacked residual blocks.

    # Arguments
        x: input tensor.
        filters: integer, filters of the bottleneck layer in a block.
        blocks: integer, blocks in the stacked blocks.
        stride1: default 2, stride of the first layer in the first block.
        name: string, stack label.

    # Returns
        Output tensor for the stacked blocks.
    """
    x = block1(x, filters, stride=stride1, name=name + '_block1')
    for i in range(2, blocks + 1):
        x = block1(x, filters, conv_shortcut=False, name=name + '_block' + str(i))
    return x

def block3(x, filters, kernel_size=3, stride=1, groups=32,
           conv_shortcut=True, name=None):
    """A residual block.

    # Arguments
        x: input tensor.
        filters: integer, filters of the bottleneck layer.
        kernel_size: default 3, kernel size of the bottleneck layer.
        stride: default 1, stride of the first layer.
        groups: default 32, group size for grouped convolution.
        conv_shortcut: default True, use convolution shortcut if True,
            otherwise identity shortcut.
        name: string, block label.

    # Returns
        Output tensor for the residual block.
    """
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    if conv_shortcut is True:
        shortcut = Conv2D((64 // groups) * filters, 1, strides=stride,
                                 use_bias=False, name=name + '_0_conv')(x)
        shortcut = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                             name=name + '_0_bn')(shortcut)
    else:
        shortcut = x

    x = Conv2D(filters, 1, use_bias=False, name=name + '_1_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_1_bn')(x)
    x = Activation('relu', name=name + '_1_relu')(x)

    c = filters // groups
    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name=name + '_2_pad')(x)
    x = DepthwiseConv2D(kernel_size, strides=stride, depth_multiplier=c,
                               use_bias=False, name=name + '_2_conv')(x)
    x_shape = backend.int_shape(x)[1:-1]
    x = Reshape(x_shape + (groups, c, c))(x)
    output_shape = x_shape + (groups, c) if backend.backend() == 'theano' else None
    x = Lambda(lambda x: sum([x[:, :, :, :, i] for i in range(c)]),
                      output_shape=output_shape, name=name + '_2_reduce')(x)
    x = Reshape(x_shape + (filters,))(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_2_bn')(x)
    x = Activation('relu', name=name + '_2_relu')(x)

    x = Conv2D((64 // groups) * filters, 1,
                      use_bias=False, name=name + '_3_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                  name=name + '_3_bn')(x)

    x = Add(name=name + '_add')([shortcut, x])
    x = Activation('relu', name=name + '_out')(x)
    return x


def stack3(x, filters, blocks, stride1=2, groups=32, name=None):
    """A set of stacked residual blocks.

    # Arguments
        x: input tensor.
        filters: integer, filters of the bottleneck layer in a block.
        blocks: integer, blocks in the stacked blocks.
        stride1: default 2, stride of the first layer in the first block.
        groups: default 32, group size for grouped convolution.
        name: string, stack label.

    # Returns
        Output tensor for the stacked blocks.
    """
    x = block3(x, filters, stride=stride1, groups=groups, name=name + '_block1')
    for i in range(2, blocks + 1):
        x = block3(x, filters, groups=groups, conv_shortcut=False,
                   name=name + '_block' + str(i))
    return x

def ResNet(stack_fn,
           preact,
           use_bias,
           model_name='resnet',
           weights='imagenet',
           input_tensor=None,
           **kwargs):
    """Instantiates the ResNet, ResNeXt architecture.

    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.

    # Arguments
        stack_fn: a function that returns output tensor for the
            stacked residual blocks.
        preact: whether to use pre-activation or not
            (True for ResNetV2, False for ResNet and ResNeXt).
        use_bias: whether to use biases for convolutional layers or not
            (True for ResNet and ResNetV2, False for ResNeXt).
        model_name: string, model name.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.


    # Returns
        A Keras model instance.the output of the model will be
                the 4D tensor output of the
                last convolutional layer

    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    global backend, keras_utils
    backend, _, _, keras_utils = get_submodules_from_kwargs(kwargs)

    if not (weights in {'imagenet', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if input_tensor is None:
       raise ValueError("input_tensor can't be None")
    
    img_input = input_tensor
    
    x = ZeroPadding2D(padding=((3, 3), (3, 3)), name='conv1_pad')(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=use_bias, name='conv1_conv')(x)

    if preact is False:
        x = BatchNormalization(axis=3, epsilon=1.001e-5,
                                      name='conv1_bn')(x)
        x = Activation('relu', name='conv1_relu')(x)

    x = ZeroPadding2D(padding=((1, 1), (1, 1)), name='pool1_pad')(x)
    x = MaxPooling2D(3, strides=2, name='pool1_pool')(x)

    x = stack_fn(x)

    if preact is True:
        x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5,
                                      name='post_bn')(x)
        x = Activation('relu', name='post_relu')(x)

    inputs = img_input

    # Create model.
    model = tf.keras.Model(inputs, x, name=model_name)

    # Load weights.
    if (weights == 'imagenet') and (model_name in WEIGHTS_HASHES):        
        file_name = model_name + '_weights_tf_dim_ordering_tf_kernels_notop.h5'
        file_hash = WEIGHTS_HASHES[model_name][1]
        weights_path = keras_utils.get_file(file_name,
                                            BASE_WEIGHTS_PATH + file_name,
                                            cache_subdir='models',
                                            file_hash=file_hash)
        model.load_weights(weights_path)
    elif weights is not None:
        model.load_weights(weights)

    return model

@keras_modules_injection
def ResNet50(weights='imagenet', input_tensor=None, **kwargs):
  def stack_fn(x):
      x = stack1(x, 64, 3, stride1=1, name='conv2')
      x = stack1(x, 128, 4, name='conv3')
      x = stack1(x, 256, 6, name='conv4')
      x = stack1(x, 512, 3, name='conv5')
      return x
  return ResNet(stack_fn, False, True, 'resnet50',
                weights, input_tensor, **kwargs)

@keras_modules_injection
def ResNeXt50(weights='imagenet', input_tensor=None, **kwargs):
    def stack_fn(x):
        x = stack3(x, 128, 3, stride1=1, name='conv2')
        x = stack3(x, 256, 4, name='conv3')
        x = stack3(x, 512, 6, name='conv4')
        x = stack3(x, 1024, 3, name='conv5')
        return x
    return ResNet(stack_fn, False, False, 'resnext50',
                  weights, input_tensor, **kwargs)

In [0]:
def get_id_from_file_path(file_path):
    return file_path.split(os.path.sep)[-1].replace('.tif', '')


def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))


def do_train_augmentations():
    return Compose([
        RandomRotate90(p=0.5),
        Transpose(p=0.5),
        Flip(p=0.5),
        OneOf([CLAHE(clip_limit=2),
              IAASharpen(),
              IAAEmboss(),
              RandomBrightnessContrast(),
              JpegCompression(),
              Blur(),
              GaussNoise()],
              p=0.5),
        HueSaturationValue(p=0.5),
        ShiftScaleRotate(shift_limit=0.15, scale_limit=0.15, rotate_limit=45, p=0.5),
        Normalize(p=1)])


def do_inference_aug():
    return Compose([Normalize(p=1)], p=1)


def data_gen(list_files, id_label_map_in, batch_size_in, img_size_in, aug_funtion):
  
    aug = aug_funtion()
    
    while True:
      
        shuffle(list_files)
        for block in chunker(list_files, batch_size_in):

            X = [cv2.resize(cv2.imread(x), (img_size_in, img_size_in)) for x in block]
            X = [aug(image=x)['image'] for x in X]

            Y = [id_label_map_in[get_id_from_file_path(x)] for x in block]

            yield np.array(X), np.array(Y)


def get_model(img_size, batch_size):
  
    weight_decay = 1e-4

    visible = Input(shape=(img_size, img_size, 3), batch_size=batch_size, dtype=tf.float32)

    conv1 = Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(visible)
    conv1_act = Activation('elu')(conv1)
    conv1_act_batch = BatchNormalization()(conv1_act)

    conv2 = Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(conv1_act_batch)
    conv2_act = Activation('elu')(conv2)
    conv2_act_batch = BatchNormalization()(conv2_act)
    conv2_act_batch_max = MaxPooling2D(pool_size=(2, 2))(conv2_act_batch)
    conv2_act_batch_max_drop = Dropout(0.2)(conv2_act_batch_max)

    conv3 = Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(
        conv2_act_batch_max_drop)
    conv3_act = Activation('elu')(conv3)
    conv3_act_batch = BatchNormalization()(conv3_act)

    conv4 = Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(conv3_act_batch)
    conv4_act = Activation('elu')(conv4)
    conv4_act_batch = BatchNormalization()(conv4_act)
    conv4_act_batch_max = MaxPooling2D(pool_size=(2, 2))(conv4_act_batch)
    conv4_act_batch_max_drop = Dropout(0.3)(conv4_act_batch_max)

    conv5 = Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(
        conv4_act_batch_max_drop)
    conv5_act = Activation('elu')(conv5)
    conv5_act_batch = BatchNormalization()(conv5_act)

    conv6 = Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(conv5_act_batch)
    conv6_act = Activation('elu')(conv6)
    conv6_act_batch = BatchNormalization()(conv6_act)
    conv6_act_batch_max = MaxPooling2D(pool_size=(2, 2))(conv6_act_batch)
    conv6_act_batch_max_drop = Dropout(0.4)(conv6_act_batch_max)
    
    flat = Flatten()(conv6_act_batch_max_drop)

    # and a logistic layer
    predictions = Dense(1, activation='sigmoid')(flat)
    
    # Create model.
    model = tf.keras.Model(visible, predictions, name='baseline')

    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.001,
    beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam'),
    loss='binary_crossentropy', metrics=['acc'])

    return model

def get_model_resnet50(img_size, batch_size):
  
    inputs = Input(shape=(img_size, img_size, 3), batch_size=batch_size, dtype=tf.float32)
  
    base_model = ResNet50(input_tensor = inputs)
    x = base_model(inputs)
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.5)(out)
    
    # and a logistic layer
    out = Dense(1, activation="sigmoid", name="3_")(out)
    
    # Create model.
    model = tf.keras.Model(inputs, out)
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.00007,
    beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam'),
    loss='binary_crossentropy', metrics=['acc'])

    return model
  
def get_model_resnext50(img_size, batch_size):
  
    inputs = Input(shape=(img_size, img_size, 3), batch_size=batch_size, dtype=tf.float32)
  
    base_model = ResNeXt50(input_tensor = inputs)
    x = base_model(inputs)
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.5)(out)
    
    # and a logistic layer
    out = Dense(1, activation="sigmoid", name="3_")(out)
    
    # Create model.
    model = tf.keras.Model(inputs, out)
    model.compile(optimizer=tf.train.AdamOptimizer(learning_rate=0.00007,
    beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam'),
    loss='binary_crossentropy', metrics=['acc'])

    return model

def plot_metric(history_in, metric_name, results_dir):
    """
    Plot a metric of model's history.
    """

    fig_acc = plt.figure(figsize=(10, 10))
    plt.plot(history_in.history[metric_name])
    plt.plot(history_in.history['val_' + metric_name])

    plt.title('model ' + metric_name)
    plt.ylabel(metric_name)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    fig_acc.savefig(results_dir+"/model_" + metric_name + ".png")

    plt.cla()
    plt.close()


def mkdir_if_not_exist(directory):

        if not os.path.exists(directory):
            try:
                os.makedirs(directory)
            except OSError as e:
                print("Error: %s - %s." % (e.filename, e.strerror))




In [0]:
#@title Configuration
#@markdown 

output_path = 'output'  #@param
train_limit = 220025  #@param {type: "number"}
test_limit = 57458  #@param {type: "number"}
epochs = 10 #@param {type:"integer"}
n_fold = 4  #@param {type: "slider", min: 2, max: 10}
number_of_tpu_core = 8  #@param {type: "slider", min: 1, max: 8}
batch_size = 88  #@param {type: "slider", min: 8, max: 128}
#@markdown ---
architecture = "ResNext50" #@param ["Baseline", "Resnet50", "ResNext50"]



In [0]:
if __name__ == '__main__':
  
    ###############
    # Configuration
    ###############
    img_size = 96
    training_batch_size = batch_size * number_of_tpu_core
    
    
    ##################
    # Data Preparation
    ##################
    
    df_train = pd.read_csv("/content/train_labels.csv")
    id_label_map = {k: v for k, v in zip(df_train.id.values, df_train.label.values)}

    labeled_files = glob('/content/train/*.tif')
    test_files = glob('/content/test/*.tif')

    print("labeled_files size :", len(labeled_files))
    print("test_files size :", len(test_files))

    df_dataset = pd.DataFrame()
    df_dataset['id'] = labeled_files[0:train_limit]
    df_dataset['label'] = df_train['label'].iloc[0:train_limit]

    ensemble_preds = np.zeros(len(test_files[0:test_limit]), dtype=np.float)

    skf = StratifiedKFold(n_splits=n_fold)

    for fold in range(0, n_fold):

        print("\nFOLD: {}".format(fold))

        mkdir_if_not_exist(os.path.join(output_path, str(fold)))

        h5_path = output_path + "/model_"+str(fold)+".h5"

        callbacks = [
            TerminateOnNaN(),
            EarlyStopping(
                monitor='val_loss',
                patience=5,
                mode='min',
                verbose=1,
                restore_best_weights=True),
            ModelCheckpoint(
                h5_path,
                monitor='val_loss',
                verbose=1,
                save_best_only=True,
                mode='min')]

        result = next(skf.split(X=df_dataset, y=df_dataset['label']), None)

        train = df_dataset.iloc[result[0]]['id'].tolist()
        val = df_dataset.iloc[result[1]]['id'].tolist()

        print("train:")
        unique, counts = np.unique(df_dataset.iloc[result[0]]['label'].values, return_counts=True)
        print(dict(zip(unique, counts)))

        print("val:")
        unique, counts = np.unique(df_dataset.iloc[result[1]]['label'].values, return_counts=True)
        print(dict(zip(unique, counts)))
        
        ######################
        # Put the model on TPU
        ######################
        
        tf.keras.backend.clear_session()
        if architecture == "Baseline":
          training_model = get_model(img_size, batch_size)
        elif architecture == "Resnet50":
          training_model = get_model_resnet50(img_size, batch_size)
        elif architecture == "ResNext50":
           training_model = get_model_resnext50(img_size, batch_size)
               
        # This address identifies the TPU we'll use when configuring TensorFlow.
        TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
        tf.logging.set_verbosity(tf.logging.INFO)
        
        # Converting the Keras model to TPU model using tf.contrib.tpu.keras_to_tpu_model
        model = tf.contrib.tpu.keras_to_tpu_model(
            training_model,
            strategy=tf.contrib.tpu.TPUDistributionStrategy(
                tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

        model.summary()

        ###########
        # Training
        ###########

        history = model.fit_generator(
            data_gen(train, id_label_map, training_batch_size, img_size, do_train_augmentations),
            validation_data=data_gen(val, id_label_map, training_batch_size, img_size, do_inference_aug),
            epochs=epochs, verbose=1,
            callbacks=callbacks,
            steps_per_epoch=len(train) // training_batch_size,
            validation_steps=len(val) // training_batch_size)

        # summarize history
#         plot_metric(history, 'loss', str(fold))
#         plot_metric(history, 'acc', str(fold))

        #################
        # Kaggle testing
        #################
        model.load_weights(h5_path)

        preds = []
        ids = []

        predict_batch_size = 128 * 8
        counter = 0
        aug = do_inference_aug()
        
        for batch in chunker(test_files[0:test_limit], predict_batch_size):
          
            print("Indexes: %i - %i" % (counter, counter + predict_batch_size))
            
            counter += predict_batch_size
            X = [aug(image=(cv2.resize(cv2.imread(x), (img_size, img_size))))['image'] for x in batch]
            ids_batch = [get_id_from_file_path(x) for x in batch]
            
            # predict_batch_size must be divisible by the number of TPU cores in use
            dummy_rows = len(batch) % number_of_tpu_core
            if dummy_rows > 0:
              for i in range(0, number_of_tpu_core - dummy_rows):
                X.append(np.zeros((img_size,img_size,3), dtype=np.float32))
            
            X = np.array(X)
            
            preds_batch = ((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).
                            ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
            
            preds += preds_batch
            ids += ids_batch

        df = pd.DataFrame({'id': ids, 'label': preds[0:-(number_of_tpu_core - dummy_rows)]})
        df.to_csv("results_"+str(fold)+".csv", index=False)
        print(df.head())

        # sum the predicted values
        ensemble_preds += np.array(preds[0:-(number_of_tpu_core - dummy_rows)], dtype=np.float)
        
        # in order to release the memory
        del history
        del model
        gc.collect()
        tf.keras.backend.clear_session()
# average the predicted values
ensemble_preds /= n_fold

df = pd.DataFrame({'id': ids, 'label': ensemble_preds.ravel()})
df.to_csv("ensemble.csv", index=False)
df.head()

labeled_files size : 220025
test_files size : 57458

FOLD: 0
train:
{0: 98181, 1: 66837}
val:
{0: 32727, 1: 22280}
INFO:tensorflow:Querying Tensorflow master (grpc://10.69.248.170:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 16420511637204072341)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6510740910710379258)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 340279278997891419)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4519146210501702623)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/ta

RuntimeError: ignored

In [0]:
ensemble_preds /= n_fold

df = pd.DataFrame({'id': ids, 'label': ensemble_preds.ravel()})
df.to_csv("ensemble.csv", index=False)
df.head()

,id,label
0,f1734ed91a06221dfbaba86f3d5448457c4679bf,0.002079
1,a43da378e3caf45599866986f5df757b11cc77b0,0.483096
2,a3af88e93ec37ab83cc8aece91faf76103201a7d,0.001331
3,4f67ca54a271d1ae865b9c1a6fac44de321b35dc,0.000094
4,ab50d6941479436cce6c022ff54e089911a812d8,0.457424
